Importing Symbols

In [13]:
import datetime
import requests
import bs4
import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb as mdb


In [7]:
def obtain_parse_wiki_snp500():
    """
    Download and parse the Wikipedia list of S&P 500 
    constituents using requests and BeautifulSoup.

    Returns a list of tuples to add to MySQL.
    """
    # Stores the current time, for the created_at record
    now = datetime.datetime.utcnow()

    # Use requests and BeautifulSoup to download the 
    # list of S&P 500 companies and obtain the symbol table
    response = requests.get(
        "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    )
    soup = bs4.BeautifulSoup(response.text, features="html.parser")

    # This selects the first table, using CSS Selector syntax
    # and then ignores the header row ([1:])
    symbolslist = soup.select('table')[0].select('tr')[1:]

    # Obtain the symbol information for each 
    # row in the S&P 500 constituent table
    symbols = []
    for i, symbol in enumerate(symbolslist):
        tds = symbol.select('td')
        symbols.append(
            (
                tds[0].select('a')[0].text.strip(),  # Ticker
                'stock', 
                tds[1].text.strip(),  # Name
                tds[3].text.strip(),  # Sector
                'USD', now, now
            ) 
        )
    return symbols


In [14]:
def insert_snp500_symbols(symbols):
    """
    Insert the S&P500 symbols into the MySQL database.
    """
    # Connect to the MySQL instance
    db_host = 'localhost'
    db_user = 'sec_user'
    db_pass = '1120'
    db_name = 'securities_master'
    con = mdb.connect(
        host=db_host, user=db_user, passwd=db_pass, db=db_name
    )

    # Create the insert strings
    column_str = """ticker, instrument, name, sector, 
                 currency, created_date, last_updated_date
                 """
    insert_str = ("%s, " * 7)[:-2]
    final_str = "INSERT INTO symbol (%s) VALUES (%s)" % \
        (column_str, insert_str)

    # Using the MySQL connection, carry out 
    # an INSERT INTO for every symbol
    with con: 
        cur = con.cursor()
        cur.executemany(final_str, symbols)

In [8]:
symbols = obtain_parse_wiki_snp500()

# Example: Print the first symbol
print(symbols[0])

# Example: Insert into your database
for symbol in symbols:
    print(symbol)  # Debugging: see each tuple before inserting
    # Insert `symbol` into your database (assuming you've set up the connection)


('MMM', 'stock', '3M', 'Industrial Conglomerates', 'USD', datetime.datetime(2024, 8, 13, 12, 45, 2, 574801), datetime.datetime(2024, 8, 13, 12, 45, 2, 574801))
('MMM', 'stock', '3M', 'Industrial Conglomerates', 'USD', datetime.datetime(2024, 8, 13, 12, 45, 2, 574801), datetime.datetime(2024, 8, 13, 12, 45, 2, 574801))
('AOS', 'stock', 'A. O. Smith', 'Building Products', 'USD', datetime.datetime(2024, 8, 13, 12, 45, 2, 574801), datetime.datetime(2024, 8, 13, 12, 45, 2, 574801))
('ABT', 'stock', 'Abbott', 'Health Care Equipment', 'USD', datetime.datetime(2024, 8, 13, 12, 45, 2, 574801), datetime.datetime(2024, 8, 13, 12, 45, 2, 574801))
('ABBV', 'stock', 'AbbVie', 'Biotechnology', 'USD', datetime.datetime(2024, 8, 13, 12, 45, 2, 574801), datetime.datetime(2024, 8, 13, 12, 45, 2, 574801))
('ACN', 'stock', 'Accenture', 'IT Consulting & Other Services', 'USD', datetime.datetime(2024, 8, 13, 12, 45, 2, 574801), datetime.datetime(2024, 8, 13, 12, 45, 2, 574801))
('ADBE', 'stock', 'Adobe Inc.'

In [19]:
import pymysql
import datetime
import bs4
import requests

# Database connection details
db_host = 'localhost'
db_user = 'sec_user'
db_pass = '1120'
db_name = 'securities_master'

# Function to obtain and parse symbols
def obtain_parse_wiki_snp500():
    """
    Download and parse the Wikipedia list of S&P 500 
    constituents using requests and BeautifulSoup.

    Returns a list of tuples to add to MySQL.
    """
    # Stores the current time, for the created_at record
    now = datetime.datetime.utcnow()

    # Use requests and BeautifulSoup to download the 
    # list of S&P 500 companies and obtain the symbol table
    response = requests.get(
        "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    )
    soup = bs4.BeautifulSoup(response.text, features="html.parser")

    # This selects the first table, using CSS Selector syntax
    # and then ignores the header row ([1:])
    symbolslist = soup.select('table')[0].select('tr')[1:]

    # Obtain the symbol information for each 
    # row in the S&P 500 constituent table
    symbols = []
    for symbol in symbolslist:
        tds = symbol.select('td')
        symbols.append(
            (
                tds[0].select('a')[0].text.strip(),  # Ticker
                'stock', 
                tds[1].text.strip(),  # Name
                tds[3].text.strip(),  # Sector
                'USD', now, now
            )
        )
    return symbols

# Main script to connect to the database and insert symbols
try:
    # Establish a database connection
    con = pymysql.connect(host=db_host, user=db_user, password=db_pass, database=db_name)
    with con.cursor() as cur:
        # Obtain and parse the S&P 500 symbols
        symbols = obtain_parse_wiki_snp500()

        # Insert the symbols into the 'symbol' table
        for symbol in symbols:
            print(symbol)  # Debugging: check the content of the symbol tuple
            cur.execute(
                "INSERT INTO symbol (ticker, instrument, name, sector, currency, created_date, last_updated_date) "
                "VALUES (%s, %s, %s, %s, %s, %s, %s)", 
                symbol
            )

        # Commit the transaction to save changes to the database
        con.commit()
        print("Symbols successfully added to the database.")
except pymysql.MySQLError as e:
    print(f"Error: {e}")
finally:
    # Ensure the connection is closed even if an error occurs
    if con:
        con.close()


('MMM', 'stock', '3M', 'Industrial Conglomerates', 'USD', datetime.datetime(2024, 8, 13, 13, 7, 16, 337196), datetime.datetime(2024, 8, 13, 13, 7, 16, 337196))
('AOS', 'stock', 'A. O. Smith', 'Building Products', 'USD', datetime.datetime(2024, 8, 13, 13, 7, 16, 337196), datetime.datetime(2024, 8, 13, 13, 7, 16, 337196))
('ABT', 'stock', 'Abbott', 'Health Care Equipment', 'USD', datetime.datetime(2024, 8, 13, 13, 7, 16, 337196), datetime.datetime(2024, 8, 13, 13, 7, 16, 337196))
('ABBV', 'stock', 'AbbVie', 'Biotechnology', 'USD', datetime.datetime(2024, 8, 13, 13, 7, 16, 337196), datetime.datetime(2024, 8, 13, 13, 7, 16, 337196))
('ACN', 'stock', 'Accenture', 'IT Consulting & Other Services', 'USD', datetime.datetime(2024, 8, 13, 13, 7, 16, 337196), datetime.datetime(2024, 8, 13, 13, 7, 16, 337196))
('ADBE', 'stock', 'Adobe Inc.', 'Application Software', 'USD', datetime.datetime(2024, 8, 13, 13, 7, 16, 337196), datetime.datetime(2024, 8, 13, 13, 7, 16, 337196))
('AMD', 'stock', 'Advance

In [18]:
import pymysql
import datetime
import bs4
import requests

# Connect to the MySQL instance
db_host = 'localhost'
db_user = 'sec_user'
db_pass = '1120'
db_name = 'securities_master'

# Function to obtain and parse symbols
def obtain_parse_wiki_snp500():
    now = datetime.datetime.utcnow()
    response = requests.get("http://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
    soup = bs4.BeautifulSoup(response.text, features="html.parser")
    symbolslist = soup.select('table')[0].select('tr')[1:]

    symbols = []
    for symbol in symbolslist:
        tds = symbol.select('td')
        symbols.append(
            (
                tds[0].select('a')[0].text.strip(),  # Ticker
                'stock', 
                tds[1].text.strip(),  # Name
                tds[3].text.strip(),  # Sector
                'USD', now, now
            )
        )
    return symbols

try:
    # Establish a database connection
    con = pymysql.connect(host=db_host, user=db_user, password=db_pass, database=db_name)
    with con.cursor() as cur:
        symbols = obtain_parse_wiki_snp500()
        for symbol in symbols:
            cur.execute("INSERT INTO symbol (column_names) VALUES (%s, %s, %s, %s, %s, %s)", symbol)
        con.commit()  # Commit the transaction
        print("Symbols successfully added to the database.")
except pymysql.MySQLError as e:
    print(f"Error: {e}")
finally:
    if con:
        con.close()  # Close the connection


TypeError: not all arguments converted during string formatting

In [15]:
if __name__ == "__main__":
    symbols = obtain_parse_wiki_snp500()
    insert_snp500_symbols(symbols)
    print("%s symbols were successfully added." % len(symbols))


503 symbols were successfully added.


In [16]:
con.commit()


NameError: name 'con' is not defined

In [1]:
from __future__ import print_function

import pandas as pd
#import MySQLdb as mdb
import pymysql


if __name__ == "__main__":
    # Obtain a database connection to the MySQL instance
    db_host = 'localhost'
    db_user = 'sec_user'
    db_pass = '1120'
    db_name = 'securities_master'
try:
    con = pymysql.connect(host=db_host, user=db_user, password=db_pass, database=db_name)
    print("Connected to the database successfully!")
except pymysql.MySQLError as e:
    print(f"Error {e.args[0]}: {e.args[1]}")

    


Connected to the database successfully!


In [2]:
    # Select all of the historic Google adjusted close data
    sql = """SELECT dp.price_date, dp.adj_close_price
             FROM symbol AS sym
             INNER JOIN daily_price AS dp
             ON dp.symbol_id = sym.id
             WHERE sym.ticker = 'GOOG'
             ORDER BY dp.price_date ASC;"""

    # Create a pandas dataframe from the SQL query
    goog = pd.read_sql_query(sql, con=con, index_col='price_date')    

    # Output the dataframe tail
    print(goog.tail())

/tmp/ipykernel_21406/1002139558.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  goog = pd.read_sql_query(sql, con=con, index_col='price_date')


Empty DataFrame
Columns: [adj_close_price]
Index: []


In [1]:
import pymysql
pymysql.install_as_MySQLdb()


In [1]:
pip install numpy

     |████████████████████████████████| 17.3 MB 2.5 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

     |████████████████████████████████| 12.4 MB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 505 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 345 kB 1.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install IPython

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install matplotlib

     |████████████████████████████████| 9.2 MB 449 kB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 409 kB/s eta 0:00:01
     |████████████████████████████████| 4.7 MB 121 kB/s eta 0:00:01
     |████████████████████████████████| 103 kB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 1.2 kB/s ta 0:00:013
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install scikit-learn


^C
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install ibPy


ERROR: Could not find a version that satisfies the requirement ibPy (from versions: none)
ERROR: No matching distribution found for ibPy
Note: you may need to restart the kernel to use updated packages.


IbPy

In [2]:

from __future__ import print_function

import datetime
from math import ceil

import bs4
import MySQLdb as mdb
import requests


In [3]:
def obtain_parse_wiki_snp500():
    """
    Download and parse the Wikipedia list of S&P500 
    constituents using requests and BeautifulSoup.

    Returns a list of tuples for to add to MySQL.
    """
    # Stores the current time, for the created_at record
    now = datetime.datetime.utcnow()

    # Use requests and BeautifulSoup to download the 
    # list of S&P500 companies and obtain the symbol table
    response = requests.get(
        "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    )
    soup = bs4.BeautifulSoup(response.text)

    # This selects the first table, using CSS Selector syntax
    # and then ignores the header row ([1:])
    symbolslist = soup.select('table')[0].select('tr')[1:]

    # Obtain the symbol information for each 
    # row in the S&P500 constituent table
    symbols = []
    for i, symbol in enumerate(symbolslist):
        tds = symbol.select('td')
        symbols.append(
            (
                tds[0].select('a')[0].text,  # Ticker
                'stock', 
                tds[1].select('a')[0].text,  # Name
                tds[3].text,  # Sector
                'USD', now, now
            ) 
        )
    return symbols


In [4]:
def insert_snp500_symbols(symbols):
    """
    Insert the S&P500 symbols into the MySQL database.
    """
    # Connect to the MySQL instance
    db_host = 'localhost'
    db_user = 'sec_user'
    db_pass = 'password'
    db_name = 'securities_master'
    con = mdb.connect(
        host=db_host, user=db_user, passwd=db_pass, db=db_name
    )

    # Create the insert strings
    column_str = """ticker, instrument, name, sector, 
                 currency, created_date, last_updated_date
                 """
    insert_str = ("%s, " * 7)[:-2]
    final_str = "INSERT INTO symbol (%s) VALUES (%s)" % \
        (column_str, insert_str)

    # Using the MySQL connection, carry out 
    # an INSERT INTO for every symbol
    with con: 
        cur = con.cursor()
        cur.executemany(final_str, symbols)



In [5]:

if __name__ == "__main__":
    symbols = obtain_parse_wiki_snp500()
    insert_snp500_symbols(symbols)
    print("%s symbols were successfully added." % len(symbols))


OperationalError: (1045, "Access denied for user 'sec_user'@'localhost' (using password: YES)")